# FourD.cpp

The goal here is to arrive at a dynamic, force directed layout written in cpp, that I can port to WebAssembly. 

I will need the following classes: 

* Vertex
* Edge
* Graph

That should be enough for now. Let's begin: 

In [1]:
#include "gmtl/gmtl/Vec.h"
#include "gmtl/gmtl/VecOps.h"
#include "gmtl/gmtl.h"
#include <vector>
#include <random>
#include <iostream>
#include <chrono> 
#include <map>
#include <sstream>

using namespace std;
using namespace std::chrono; 
using namespace gmtl;

## Constants

In [2]:
class Constants {
  public:
    static constexpr float repulsion = 25.0;
    static constexpr float epsilon = 0.1;
    static constexpr float inner_distance = 0.36;
    static constexpr float attraction = 0.05;
    static constexpr float friction = 0.60;
    static constexpr float gravity = 10;
  
    static constexpr float min_start_pos = -1.0f;
    static constexpr float max_start_pos = 1.0f;
};

## The Randomator class

In [3]:
template <typename T>
class Randomator {
  public:
    static inline random_device r;
    static inline default_random_engine * engine = new default_random_engine(Randomator::r());
    uniform_real_distribution<T> dist;
  
    Randomator(T min, T max){
      this->dist = uniform_real_distribution<T>(min, max);
    }

    T get(){
      return this->dist(*Randomator::engine);
    }
};



In [4]:
(new Randomator<float>( -1, 1))->get()

(float) -0.561327f


In [5]:
float length(gmtl::Vec3f v){
  return sqrt(v[0]*v[0] + v[1]*v[1] + v[2]*v[2]);
};


In [6]:
auto r = Randomator<float>(-1.0f, 1.0f);

gmtl::Vec3f vec1(r.get(), r.get(), r.get());
gmtl::Vec3f vec2(r.get(), r.get(), r.get());

In [7]:
cout << vec1 << " " << vec2 << endl;
cout << vec1 - vec2 << endl;

(-0.228382, -0.419222, 0.12854) (0.368282, -0.289008, 0.638888)
(-0.596664, -0.130214, -0.510348)


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fdb33373640


## The Vertex class

In [8]:
template <class T>
class Vertex {
  public:
    static inline Randomator<float> r = Randomator<float>(-1.0f, 1.0f);
    static inline int _id = 0;
  
    Vertex(T userdata){
      data = userdata;
      position = gmtl::Vec3f(Vertex::r.get(), Vertex::r.get(), Vertex::r.get());
      id = Vertex::_id++;
    }

    int id;
    T data;
    gmtl::Vec3f position;
    gmtl::Vec3f velocity;
    gmtl::Vec3f acceleration;
  
    gmtl::Vec3f repulsion_forces;
    gmtl::Vec3f attraction_forces;
  
    static gmtl::Vec3f pairwise_repulsion(const gmtl::Vec3f& one, const gmtl::Vec3f& other){
      gmtl::Vec3f diff = one - other;
      // gmtl::Vec3f diff = *(this->position) - *(other->position);
      float abs_diff = length(diff);
      return  (Constants::repulsion / 
               ((Constants::epsilon + abs_diff)*(Constants::epsilon + abs_diff)) * 
               (diff / abs_diff));
    }
  
    bool operator==(const Vertex<T>& other){
      return other.id == id;
    }
  
    string toString(){
      stringstream ss;
      ss << "Vertex " << id;
      return ss.str();
    }
};

## Testing the Pairwise Repulsion Function

In [9]:
Vertex<int> v1(1);
Vertex<int> v2(2);
cout << Vertex<int>::pairwise_repulsion(v1.position, v2.position) << std::endl;

(-3.86804, 7.35013, -3.12289)


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fdb33373640


## The Edge Class

In [10]:
template <class T>
class Edge {
  public:
    static inline int _id = 0;
  
    Edge(Vertex<T>* _source, Vertex<T>* _target){
      source = _source;
      target = _target;
      id = Edge::_id++;
    };

    int id;
    Vertex<T>* source;
    Vertex<T>* target;
    bool directed = false;
  
    string toString(){
      stringstream ss;
      ss << "Edge " << id;
      return ss.str();
    }
};

## The Barnes Hut Tree

In [11]:
template <class T>
class BarnesHutNode3 {
  public:
    vector<Vertex<T>> inners;
    map<string, BarnesHutNode3<T>*> outers;
    gmtl::Vec3f center_sum;
    int count = 0;
  
    gmtl::Vec3f center(){
      return this->center_sum / (float)this->count;
    }
  
    void place_inner(Vertex<T>& vertex){
      this->inners.push_back(vertex);
      this->center_sum += vertex.position;
    }
  
    void place_outer(Vertex<T>& vertex){
      string octant = this->get_octant(vertex.position);
      // this->outers.insert(make_pair(octant, new BarnesHutNode3<T>()));
      this->outers[octant] = new BarnesHutNode3<T>();
      this->outers[octant]->insert(vertex);
    }
  
    void insert(Vertex<T>& vertex){
      if(this->inners.size() == 0){
        this->place_inner(vertex);
      }else{
        gmtl::Vec3f center = this->center();
        gmtl::Vec3f pos = vertex.position;
        float distance = sqrt((center[0] - pos[0])*(center[0] - pos[0]) + 
                             (center[1] - pos[1])*(center[1] - pos[1]) +
                             (center[2] - pos[2])*(center[2] - pos[2]));
        
        if(distance <= Constants::inner_distance){
          this->place_inner(vertex);
        }else{
          this->place_outer(vertex);
        }
      }
      
      this->count++;
    }
  
    string get_octant(gmtl::Vec3f& position){
      gmtl::Vec3f center = this->center();
      string x = center[0] < position[0] ? "l" : "r";
      string y = center[1] < position[1] ? "u" : "d";
      string z = center[2] < position[2] ? "i" : "o";
      return x+y+z;
    }
  
    void estimate(Vertex<T>& vertex, gmtl::Vec3f& force, gmtl::Vec3f (*force_fn)(const gmtl::Vec3f& p1, const gmtl::Vec3f& p2)){
      if(find(this->inners.begin(), this->inners.end(), vertex) != this->inners.end()){ // todo: make better, maintain a set or something
        for(auto i=0; i<this->inners.size(); i++){
          if(this->inners[i].id != vertex.id){
            gmtl::Vec3f f = force_fn(vertex.position, this->inners[i].position);
            force += f;
          }
        }
      }else{
        gmtl::Vec3f f = force_fn(vertex.position, this->center()) * (float)this->inners.size();
        force += f;
      }
      
      for(auto &it : this->outers){
        this->outers[it.first]->estimate(vertex, force, force_fn);
      }
    }
  
    string toString(){
      return "BarnesHutNode3";
    }
  
    unsigned int size(){
      return this->count;
    }
};

## The Graph Class

In [12]:
template <class T>
class Graph {
  public:
    void add_vertex(Vertex<T> vertex){
      V.push_back(vertex);
    }
  
    void add_edge(Edge<T> edge){
      E.push_back(edge);
    }
  
    void remove_vertex(Vertex<T> vertex){
      V.erase(vertex);
    }
  
    void remove_edge(Edge<T> edge){
      E.erase(edge);
    }
  
    vector<Vertex<T>> V;
    vector<Edge<T>> E;
  
    void layout(){
      // calculate repulsions
      
      BarnesHutNode3<T> tree = BarnesHutNode3<T>();
      for(Vertex<T> vertex : this->V){
        tree.insert(vertex);
      }
      for(Vertex<T> vertex : this->V){
        vertex.repulsion_forces = gmtl::Vec3f();
        tree.estimate(
          vertex,
          vertex.repulsion_forces,
          &Vertex<T>::pairwise_repulsion);
      }
      
      // calculate attractions 
      for(Edge<T> edge : this->E){
        gmtl::Vec3f attraction = (edge.source->position - edge.target->position) * (-1 * Constants::attraction);
        if(edge.directed){
          gmtl::Vec3f sp = edge.source->position;
          gmtl::Vec3f tp = edge.target->position;
          
          float distance = sqrt((sp[0] - tp[0])*(sp[0] - tp[0]) + 
                                      (sp[1] - tp[1])*(sp[1] - tp[1]) +
                                      (sp[2] - tp[2])*(sp[2] - tp[2]));
          gmtl::Vec3f gravity = gmtl::Vec3f(0.0f, Constants::gravity/distance, 0.0f);
          edge.source->attraction_forces -= attraction;
          edge.target->attraction_forces += attraction;
        }
      }
      
      // update vertices
      for(Vertex<T> vertex : this->V){
        gmtl::Vec3f friction = vertex.velocity * Constants::friction;
        vertex.acceleration += vertex.repulsion_forces - vertex.attraction_forces - friction;
        vertex.velocity += vertex.acceleration;
        vertex.position += vertex.velocity;
      }
    }
};

## A quick test of the graph

In [13]:
Graph<int> graph;

The graph instantiates.

In [14]:
cout << "Empty graph layout test... ";
graph.layout();
cout << "done." << endl;

Empty graph layout test... done.


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fdb33373640


The layout function runs when called on an empty graph.

In [15]:
#include <iostream>
#include <chrono>
using namespace std;
using namespace std::chrono;

int NUM_VERTICES = 10;
int NUM_EDGES = 20;

auto start = high_resolution_clock::now(); 
for(int i=0; i<NUM_VERTICES; i++){
  graph.add_vertex(Vertex<int>(i));
}

for(int n=0; n<NUM_EDGES; ++n){
  graph.add_edge(Edge<int>(&graph.V[random() % graph.V.size()], &graph.V[random() % graph.V.size()]));
}
auto stop = high_resolution_clock::now(); 

cout << "Added " << NUM_VERTICES << " vertices in " << duration<double>(stop - start).count() << " seconds." << endl; 

Added 10 vertices in 2.5878e-05 seconds.


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fdb33373640


In [16]:
cout << graph.V.size() << " vertices" << endl;
cout << graph.E.size() << " edges" << endl;

10 vertices
20 edges


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fdb33373640


In [17]:
#include <iostream>
#include <chrono>
using namespace std;
using namespace std::chrono;

auto start2 = high_resolution_clock::now();
gmtl::Vec3f all;

for(auto i=0; i<graph.V.size(); i++){
  for(auto j=0; j<graph.V.size(); j++){
    if(i<j){
      all += Vertex<int>::pairwise_repulsion(graph.V[i].position, graph.V[j].position);
    }
  }
}
auto stop2 = high_resolution_clock::now();
cout << "Took " << duration<double>(stop2 - start2).count() << "s : " << " calculate "<< (all / (float)graph.V.size()) << endl;

Took 1.6478e-05s :  calculate (-13.0254, 8.84873, -14.778)


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fdb33373640


## Testing the insert and estimate functions

In [18]:
BarnesHutNode3<int> tree;

The tree instantiates.

In [19]:
using namespace std;
using namespace std::chrono;

auto start3 = high_resolution_clock::now();
for(auto i=0; i<graph.V.size(); i++){
  tree.insert(graph.V[i]);
}
auto stop3 = high_resolution_clock::now();

cout << "Took " << duration<double>(stop3 - start3).count() << "s to insert " << graph.V.size() << " vertices." << endl;

Took 3.8274e-05s to insert 10 vertices.


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fdb33373640


Tree insertion successful.

In [20]:
using namespace std;
using namespace std::chrono;

for(auto vertex : graph.V){
  auto start4 = high_resolution_clock::now();
  gmtl::Vec3f force;
  tree.estimate(vertex, force, &Vertex<int>::pairwise_repulsion);
  auto stop4 = high_resolution_clock::now();
  cout << "Took " << duration<double>(stop4 - start4).count() << "s to estimate " << force << endl;
}

Took 1.6689e-05s to estimate (-41.0847, 33.4861, -60.7888)
Took 5.232e-06s to estimate (11.5379, 79.3279, 69.9932)
Took 4.645e-06s to estimate (-20.0824, 13.4598, -78.6327)
Took 4.446e-06s to estimate (-6.13158, 22.8258, 5.08472)
Took 4.564e-06s to estimate (44.4381, -74.0126, 5.13635)
Took 4.296e-06s to estimate (105.859, 3.66134, -85.4243)
Took 4.185e-06s to estimate (-46.8627, -53.0464, 45.2093)
Took 4.164e-06s to estimate (-67.0642, 22.1356, 15.4438)
Took 4.086e-06s to estimate (83.6873, 59.9389, 29.3155)
Took 4.107e-06s to estimate (53.5647, -21.2954, 53.8714)


In [21]:
long rs[] = {random(), random()};
rs

(long [2]) { 149798315, 2038664370 }


## Testing graph.layout()

In [22]:
#include <iostream>
#include <chrono>
#include <iostream>
using namespace std;
using namespace std::chrono;

NUM_VERTICES = 1000;
NUM_EDGES = NUM_VERTICES * 3;

time_point start5 = high_resolution_clock::now();
Graph<int> g = Graph<int>();
for(int i=0; i<NUM_VERTICES; i++){
  g.add_vertex(Vertex<int>(i));
}
for(int i=0; i<NUM_EDGES; i++){
  g.add_edge(Edge<int>(&g.V[random() % g.V.size()], &g.V[random() % g.V.size()]));
}
g.layout();
time_point stop5 = high_resolution_clock::now();

cout << "g.layout() took " << duration<double>(stop5 - start5).count() << "s with " << g.V.size() << " vertices and " << g.E.size() << " edges." << endl;

g.layout() took 0.0148651s with 1000 vertices and 3000 edges.


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7fdb33373640
